In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Correlation Analysis/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Correlation Analysis/Output')
cur_date = "121222"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_121222.xlsx", sheet = 2))
#acs_metadata = data.frame(read_excel("ACS_Data_120722.xlsx"))
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_121222.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(acs_df)
#head(acs_metadata)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.299999999999997,21.9,78.099999999999994,15.1,61,28.5,⋯,20.8,14.6,26.4,28,54.8,0,45.3,0,31.3,13.9
2,14000US37001020200,202.00,Alamance,78.3,31,29.1,70.900000000000006,11.2,51.9,29.2,⋯,33,19.2,27.3,37.799999999999997,44.6,0,0,82.7,16.2,39
3,14000US37001020300,203.00,Alamance,77.8,40.700000000000003,27.1,72.900000000000006,18.899999999999999,45.2,28.9,⋯,17.3,22,18,28.3,21,NA,10.9,36.4,26.7,20.100000000000001
4,14000US37001020400,204.00,Alamance,78.099999999999994,30.5,31.4,68.599999999999994,10.1,19.600000000000001,59,⋯,26.8,14,30,33.299999999999997,19.899999999999999,NA,69,100,67.2,26.6
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.099999999999994,13.3,⋯,12.2,7.1,12.3,14.8,18.899999999999999,0,0,53.1,25.7,10
6,14000US37001020502,205.02,Alamance,99,32.1,30,70,8.9,40.700000000000003,35.700000000000003,⋯,22.3,19.2,17.7,31,27.8,100,14.2,19.5,14.7,25


,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,14000US37001020100,201.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
2,14000US37001020200,202.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
3,14000US37001020301,203.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%
4,14000US37001020302,203.02,Alamance,278233.79700000002,92.237034639041696,1,0-20%
5,14000US37001020400,204.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
6,14000US37001020501,205.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%


# What sociodemographic variables are associated with wildfire hazard potential?

Wildfire hazard potential (WHP) is an index that quantifies the relative potential for wildfire that may be difficult to control, used as a measure to help prioritize where fuel treatments may be needed [ref](https://www.fs.usda.gov/rmrs/datasets/wildfire-hazard-potential-united-states-270-m-version-2020-3rd-edition). Pearson's correlation coefficients will be calculated to see if there are associations between each variable and WHP.

In [3]:
# lot's of missing data
dim(full_join(acs_df, wildfire_hazard_df))
dim(inner_join(acs_df, wildfire_hazard_df))

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 3137   34

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 1730   34

In [6]:
acs_whp_df = inner_join(acs_df, wildfire_hazard_df)

# need to make some of the cols numeric
acs_whp_df[,4:33] = apply(acs_whp_df[,4:33], 2, as.numeric)
head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 4:33], 2, 

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0,45.3,0.0,31.3,13.9,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0,0.0,82.7,16.2,39.0,278233.8,92.23703,1,0-20%
3,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,NA,69.0,100.0,67.2,26.6,278233.8,92.23703,1,0-20%
4,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0,0.0,53.1,25.7,10.0,278233.8,92.23703,1,0-20%
5,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100,14.2,19.5,14.7,25.0,278233.8,92.23703,1,0-20%
6,14000US37001020601,206.01,Alamance,97.3,51.7,17.5,82.5,27.8,96.9,0.9,⋯,0.0,0,100.0,0.0,6.7,3.6,278233.8,92.23703,1,0-20%


# Random Forest Imputation
Generating missing values using random forest (RF).

In [8]:
# imputing sociodemographic data using RF
random_forest_imputation = function(dataset){
    imputed_RF_object = missForest(as.matrix(dataset[,4:30]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:3], imputed_RF_df, dataset[,31:34])) 
    
    return(imputed_final_df)
}

# calling function
imputed_df = random_forest_imputation(acs_whp_df)
head(imputed_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0.000,45.3,0.0,31.3,13.9,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0.000,0.0,82.7,16.2,39.0,278233.8,92.23703,1,0-20%
3,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,29.376,69.0,100.0,67.2,26.6,278233.8,92.23703,1,0-20%
4,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0.000,0.0,53.1,25.7,10.0,278233.8,92.23703,1,0-20%
5,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100.000,14.2,19.5,14.7,25.0,278233.8,92.23703,1,0-20%
6,14000US37001020601,206.01,Alamance,97.3,51.7,17.5,82.5,27.8,96.9,0.9,⋯,0.0,0.000,100.0,0.0,6.7,3.6,278233.8,92.23703,1,0-20%


In [30]:
number_of_socio_variables = 27 # number of sociodemographic variables

# creating a function to run pearson tests
get_pearson_correlations = function(df, number_of_var){
    
    # creating an empty df
    corr_df = data.frame()
    for (i in 1:number_of_var){
        if (i <= number_of_var){
             # pearson's test
            pearson_test = cor.test(df[,i + 3], df$Wildfire_Hazard_Potential_Mean, method = c('pearson'))

            # remembering values of interest
            values_df = data.frame(colnames(df[i + 3]), pearson_test$estimate[[1]], pearson_test$p.value[[1]])
            corr_df = rbind(corr_df, values_df)
        }
    }
    
    # adding col names
    colnames(corr_df) = c("Variable", "Estimate", "P Value")

    # calculating padj values
    corr_df$`P Adjust` = p.adjust(as.numeric(as.character(corr_df$`P Value`)), method = "fdr")
    
    # arranging from most to least significant
    corr_df = corr_df %>%
        arrange(`P Adjust`)
    
    return(corr_df)
}

# calling function
pearson_df = get_pearson_correlations(imputed_df, number_of_socio_variables)
head(pearson_df)

,Variable,Estimate,P Value,P Adjust
,<chr>,<dbl>,<dbl>,<dbl>
1,Race_Indigenous,0.13222599,3.452424e-08,9.321545e-07
2,Race_Mixed,0.11104578,3.708881e-06,5.006989e-05
3,Sex_Ratio,0.09470834,8.047923e-05,7.243130e-04
4,Race_Asian,-0.08717499,2.855339e-04,1.927354e-03
5,English_Speakers_Only,0.08302196,5.510567e-04,2.975706e-03
6,Non_Proficient_English_Speakers,-0.07766856,1.232859e-03,5.547865e-03


Doing a similar analysis this time seeing if the quantile WHP are associated with sociodemographic variables.